In [55]:
from dotenv import load_dotenv
load_dotenv()
from langchain_core.prompts  import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
chatmodel=ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [80]:
import os
from openai import OpenAI
from langchain_core.runnables import Runnable

# Initialize Hugging Face router client
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.environ["HF_TOKEN"],
)

class HuggingFaceChatModel(Runnable):
    """LangChain-compatible Hugging Face ChatModel using OpenAI-compatible API."""

    def __init__(self, model_name="openai/gpt-oss-safeguard-20b:groq"):
        self.model_name = model_name

    def invoke(self, inputs, config=None, **kwargs):
        """Run a single prompt through the model."""
        if isinstance(inputs, dict):
            prompt = inputs.get("input") or inputs.get("prompt")
        else:
            prompt = str(inputs)

        if not prompt or not prompt.strip():
            raise ValueError("Prompt text is empty or invalid.")

        try:
            completion = client.chat.completions.create(
                model=self.model_name,
                messages=[{"role": "user", "content": prompt}],
            )
            return completion.choices[0].message.content
        except Exception as e:
            return f"Error: {e}"


In [81]:
chatmodel2=HuggingFaceChatModel()

In [86]:
prompt=ChatPromptTemplate.from_template("tell me a curious fact about {player}")

output_parser=StrOutputParser()

In [87]:
chain =  prompt | chatmodel.bind(stop={"Ronaldo"}) |  output_parser
chain.invoke({"player":"Ronaldo"})

"Here's a curious fact about **Cristiano "

# Combining Lcel chains

In [88]:
prompt=ChatPromptTemplate.from_template("tell me a senetence about {politician}")
chain2=prompt | chatmodel | output_parser

chain2.invoke({"politician":"Narendra Modi"})

'Narendra Modi is the current Prime Minister of India.'

# Combining Chain

In [90]:
historian_prompt=ChatPromptTemplate.from_template("Was {politician} positive for humanity")

composed_chain={"politician":chain2} | historian_prompt | chatmodel | StrOutputParser()

answer=composed_chain.invoke({"politician":"Narendra Modi"})

In [91]:
print(answer)

Yes, Narendra Modi is the current Prime Minister of India, serving in that role since May 26, 2014.

Assessing whether his tenure has been "positive for humanity" is a complex question that elicits diverse perspectives, both within India and internationally. There isn't a simple "yes" or "no" answer, as different people weigh various factors differently.

Here's a breakdown of arguments often made, both in favor and critically:

**Arguments for Positive Impact (from supporters and some analyses):**

1.  **Economic Development and Poverty Alleviation:**
    *   **Financial Inclusion:** Schemes like Jan Dhan Yojana have brought millions into the formal banking system.
    *   **Sanitation:** Swachh Bharat Abhiyan (Clean India Mission) significantly improved sanitation coverage, reducing open defecation.
    *   **Access to Utilities:** Ujjwala Yojana provided LPG connections to millions of rural households, reducing reliance on polluting fuels. Saubhagya scheme aimed for universal househ